## 12.10 定义一个 Actor 任务

Actor模式是一种并发模型，与另一种模型共享内存完全相反，Actor模型share nothing。所有的线程(或进程)通过消息传递的方式进行合作，这些线程(或进程)称为Actor。  
MapReduce就是一种典型的Actor模式.  
到了分布式系统时代，工厂已经用流水线了，每个人都有明确分工，这就是Actor模式。每个线程都是一个Actor，这些Actor不共享任何内存，所有的数据都是通过消息传递的方式进行的。

,一个 actor 就是一个并发执
行的任务,只是简单的执行发送给它的消息任务。响应这些消息时,它可能还会给其他
actor 发送更进一步的消息。actor 之间的通信是单向和异步的。因此,消息发送者不知
道消息是什么时候被发送,也不会接收到一个消息已被处理的回应或通知。

## 12.11 实现消息发布/订阅模型 

要实现发布/订阅的消息通信模式,你通常要引入一个单独的“交换机”或“网关”
对象作为所有消息的中介。也就是说,**不直接将消息从一个任务发送到另一个(actor模式),而是将
其发送给交换机,然后由交换机将它发送给一个或多个被关联任务。**

使用一个交换机可以简化大部分涉及到线程通信的工作。无需去写通过多进
程模块来操作多个线程,你只需要使用这个交换机来连接它们。某种程度上,这个就跟
日志模块的工作原理类似。实际上,它可以轻松的解耦程序中多个任务。

其次,交换机广播消息给多个订阅者的能力带来了一个全新的通信模式。例如,你
可以使用多任务系统、广播或扇出。你还可以通过以普通订阅者身份绑定来构建调试和
诊断工具。

## 12.14 在 Unix 系统上面启动守护进程

In [ ]:
#!/usr/bin/env python3
import sys
import os
import atexit
import signal


def daemonize(pidfile, *, stdin='/dev/null',
              stdout='/dev/null',
              stderr='/dev/null'):
    if os.path.exists(pidfile):
        raise RuntimeError('Already running')
  
    # First fork (detaches from parent)
    try:
    if os.fork() > 0:
        raise SystemExit(0)     # Parent exit
    except OSError as e:
        raise RuntimeError('fork #1 failed.')
        
    os.chdir('/')
    os.umask(0)
    os.setsid()
    
    # Second fork (relinquish session leadership)
    try:
        if os.fork() > 0:
            raise SystemExit(0)
    except OSError as e:
        raise RuntimeError('fork #2 failed.')
        
    # Flush I/O buffers
    sys.stdout.flush()
    sys.stderr.flush()
    
    # Replace file descriptors for stdin, stdout, and stderr
    with open(stdin, 'rb', 0) as f:
        os.dup2(f.fileno(), sys.stdin.fileno())
    with open(stdout, 'ab', 0) as f:
        os.dup2(f.fileno(), sys.stdout.fileno())
    with open(stderr, 'ab', 0) as f:
        os.dup2(f.fileno(), sys.stderr.fileno())
    # Write the PID fileb
    with open(pidfile,'w') as f:
        print(os.getpid(),file=f)
        
    # Arrange to have the PID file removed on exit/signal
    atexit.register(lambda: os.remove(pidfile))
    
    # Signal handler for termination (required)
    def sigterm_handler(signo, frame):
        raise SystemExit(1)        
    signal.signal(signal.SIGTERM, sigterm_handler)
    

def main():
    import time
    sys.stdout.write('Daemon started with pid {}\n'.format(os.getpid()))
    while True:
        sys.stdout.write('Daemon Alive! {}\n'.format(time.ctime()))
        time.sleep(10)
        
        
if __name__ == '__main__':
    PIDFILE = '/tmp/daemon.pid'
    
    if len(sys.argv) != 2:
        print('Usage: {} [start|stop]'.format(sys.argv[0]), file=sys.stderr)
        raise SystemExit(1)

    if sys.argv[1] == 'start':
        try:
            daemonize(PIDFILE, stdout='/tmp/daemon.log',stderr='/tmp/dameon.log')
        except RuntimeError as e:
            print(e, file=sys.stderr)
            raise SystemExit(1)
            
        main()
        
    elif sys.argv[1] == 'stop':
        if os.path.exists(PIDFILE):
            with open(PIDFILE) as f:
                os.kill(int(f.read()), signal.SIGTERM)
        else:
            print('Not running', file=sys.stderr)
            raise SystemExit(1)
    else:
        print('Unknown command {!r}'.format(sys.argv[1]), file=sys.stderr)
        raise SystemExit(1)


daemonize() 函数只接受关键字参数,这样的话可选参数在被使用时就更清晰了

在子进程变成孤儿后,调用 `os.setsid()` 创建了一个全新的进程会话,并设置子
进程为首领。它会**设置这个子进程为新的进程组的首领,并确保不会再有控制终端。**

它需要将守护进程同终端分离开并确保信号机制对它不起
作用。调用` os.chdir() 和 os.umask(0) `改变了当前工作目录并重置文件权限掩码。修
改目录通常是个好主意,因为这样可以使得它不再工作在被启动时的目录。

一旦守护进程被正确的分离,它会重新初始化标准 I/O 流指向用户指定的文件。

这一部分有点难懂。跟标准 I/O 流相关的文件对象的引用在解释器中多个地方被找到(sys.stdout, sys.__stdout__ 等)。
**仅仅简单的关闭 sys.stdout 并重新指定它是行不
通的,因为没办法知道它是否全部都是用的是 sys.stdout 。这里,我们打开了一个单
独的文件对象,并调用 os.dup2() ,用它来代替被 sys.stdout 使用的文件描述符。这
样,sys.stdout 使用的原始文件会被关闭并由新的来替换。还要强调的是任何用于文
件编码或文本处理的标准 I/O 流还会保留原状。**


守护进程的一个通常实践是在一个文件中写入进程 ID,可以被其他程序后面使用
到。 daemonize() 函数的最后部分写了这个文件,但是在程序终止时删除了它

**一个对于 SIGTERM 的信号处理器的定义同样需要被优雅的关闭。**信号处理器简单的抛出了 SystemExit() 异常。
 或许这一步看上去没必要,但是没有它,终止信号会使得不执行 atexit.register()注册的清理操作的时候就杀掉了解释器。一个杀掉进程的例子代码可以在程序最后的stop 命令的操作中看到。